In [9]:
## Split data
# import pandas as pd
# import numpy as np

# df = pd.read_csv('../Raw Data/customer_order.csv')

# count = 0
# for i in range(5):
#     if count + df.shape[0]//5 < df.shape[0]:
#         df.iloc[count:count + df.shape[0]//5, :].to_csv(f'../Raw Data/customer_order_{i}.csv', index=False)
#         count += df.shape[0]//5
#     else:
#         df.loc[count:, :].to_csv(f'../Raw Data/customer_order_{i}.csv', index=False)

In [3]:
from datetime import datetime 
import pyspark.sql.functions as sparkFunction
import pyspark.sql.types as dataType
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

HOSTNAME = "localhost"
PORT = 1433
USERNAME = "sa"
PASSWORD = "Longhandsome123"
# DB_NAME = "demo"

def createUrl(dbName, hostname=HOSTNAME, port=PORT, username=USERNAME, password=PASSWORD):
    return f"jdbc:sqlserver://{hostname}:{port};database={dbName};user={username};password={password}"
    
## day number 0
## day number 1
dayNum = 1
fileName = f'../Raw Data/customer_order_{dayNum}.csv'

df = spark.read.csv(fileName, header=True, inferSchema=True)
# trước khi xử lý datetime
df.printSchema()

def convertToDateType(dateString):
    return datetime.strptime(dateString, '%m/%d/%y')

str2Date = sparkFunction.udf(f=convertToDateType, returnType=dataType.DateType())
df = df.withColumn('Ship Date', str2Date('Ship Date'))
df = df.withColumn('Order Date', str2Date('Order Date'))
# sau khi xử lý datetime
df.printSchema()

# append to source
df.write.jdbc(createUrl(dbName='SourceSystem'), table='customer_order', mode='append')
print(f'Write {df.count()} records into db')

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: date (nullable = true)
 |-- Ship Date: date 